In [1]:
import pandas as pd, numpy as np
from pathlib import Path
import sys
import os

# 프로젝트 루트 경로 추가
ROOT_DIR = os.path.dirname(os.path.abspath("."))
DATA_DIR = os.path.join(ROOT_DIR, "data")

DATA_DIR

'/Users/tykim/Desktop/work/SNU_bigdata_fintech_2025/hotel_booking_cancel_classification/data'

In [2]:
RAW = Path(os.path.join(DATA_DIR, "raw"))
OUT = Path(os.path.join(DATA_DIR, "processed"))
OUT.mkdir(parents=True, exist_ok=True)


df = pd.read_csv(RAW / "hotel_bookings.csv")  # Kaggle 파일명

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [3]:
df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')

# 결측치 처리

In [4]:
for col in df.columns:
    if df[col].isna().sum() > 0:
        print(
            f"{col} 컬럼의 결측치 개수: {df[col].isna().sum()}, 타입: {df[col].dtype}, 비율: {(df[col].isna().sum() / len(df)) * 100}"
        )

children 컬럼의 결측치 개수: 4, 타입: float64, 비율: 0.003350364352123293
country 컬럼의 결측치 개수: 488, 타입: object, 비율: 0.40874445095904177
agent 컬럼의 결측치 개수: 16340, 타입: float64, 비율: 13.686238378423655
company 컬럼의 결측치 개수: 112593, 타입: float64, 비율: 94.30689337465449


In [5]:
df['children'] = df["children"].fillna(0)
df["agent"] = df["agent"].fillna(0)
df["company"] = df["company"].fillna(0)

In [6]:
df['children'].isna().sum()

0

# 이상치 처리

In [ ]:
# 성인+어린이+영아가 0인 행 제거
df = df[(df["adults"] + df["children"] + df["babies"]) > 0].copy()

df.shape

(119210, 32)

In [8]:
df["arrival_date_day_of_month"]

0          1
1          1
2          1
3          1
4          1
          ..
119385    30
119386    31
119387    31
119388    31
119389    29
Name: arrival_date_day_of_month, Length: 119210, dtype: int64

In [9]:
# 날짜 조합
# arrival_date_year, arrival_date_month(문자), arrival_date_day_of_month가 제공됨
month_map = {
    m: i
    for i, m in enumerate(
        [
            "January",
            "February",
            "March",
            "April",
            "May",
            "June",
            "July",
            "August",
            "September",
            "October",
            "November",
            "December",
        ],
        start=1,
    )
}

# month 숫자로 변환
df["arrive_month_num"] = df["arrival_date_month"].map(month_map)

# 날짜 조합
df["arrival_date"] = pd.to_datetime(
    dict(
        year=df["arrival_date_year"],
        month=df["arrive_month_num"],
        day=df["arrival_date_day_of_month"],
    ),
    errors="coerce",
)


print(month_map)
df["arrival_date"]

{'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}


0        2015-07-01
1        2015-07-01
2        2015-07-01
3        2015-07-01
4        2015-07-01
            ...    
119385   2017-08-30
119386   2017-08-31
119387   2017-08-31
119388   2017-08-31
119389   2017-08-29
Name: arrival_date, Length: 119210, dtype: datetime64[ns]

In [10]:
# 체류 관련

# 체류 일수
df["stay_nights"] = df["stays_in_week_nights"] + df["stays_in_weekend_nights"]

# 총 인원 수
df["total_guests"] = df["adults"] + df["children"] + df["babies"]

# 주말 도착 여부
df["is_weekend_arrival"] = df["arrival_date"].dt.dayofweek.isin([5, 6]).astype(int)

# 리드 타임
df["lead_time"] = df["lead_time"].astype(int)  # 이미 제공

In [11]:
# 카테고리 소수화: country 너무 희소 → 빈도상위 + 기타
top_countries = df["country"].value_counts().head(20).index
df["country_top"] = np.where(df["country"].isin(top_countries), df["country"], "OTHER")

df["country_top"].isna().sum()

0

In [12]:
# 불필요/누수 위험 컬럼 제거
drop_cols = [
    "reservation_status",
    "reservation_status_date",  # 누수
    "arrival_date_year",
    "arrival_date_month",
    "arrival_date_day_of_month",
    "arrival_date_week_number",
    "arrive_month_num",
    "assigned_room_type",  # 옵션: 기본 제외
    "company",
    "agent",
    "country",  # 정규화 전 원컬럼
]
for c in drop_cols:
    if c in df.columns:
        df.drop(columns=c, inplace=True, errors="ignore")


df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119210 entries, 0 to 119389
Data columns (total 27 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   hotel                           119210 non-null  object        
 1   is_canceled                     119210 non-null  int64         
 2   lead_time                       119210 non-null  int64         
 3   stays_in_weekend_nights         119210 non-null  int64         
 4   stays_in_week_nights            119210 non-null  int64         
 5   adults                          119210 non-null  int64         
 6   children                        119210 non-null  float64       
 7   babies                          119210 non-null  int64         
 8   meal                            119210 non-null  object        
 9   market_segment                  119210 non-null  object        
 10  distribution_channel            119210 non-null  object      

In [13]:
# 특징(수치 + 범주)
cat_cols = [
    "hotel",
    "meal",
    "market_segment",
    "distribution_channel",
    "reserved_room_type",
    "deposit_type",
    "customer_type",
    "country_top",
]
num_cols = [
    "lead_time",
    "stay_nights",
    "adults",
    "children",
    "babies",
    "total_guests",
    "previous_cancellations",
    "previous_bookings_not_canceled",
    "booking_changes",
    "days_in_waiting_list",
    "adr",
    "is_repeated_guest",
    "is_weekend_arrival",
]
use_cols = cat_cols + num_cols + ["arrival_date"]  # arrival_date는 스플릿용

# 타깃
y = df["is_canceled"].astype(int)

X = df[use_cols].copy()

print(len(use_cols))

22


In [14]:
X

,hotel,meal,market_segment,distribution_channel,reserved_room_type,deposit_type,customer_type,country_top,lead_time,stay_nights,...,babies,total_guests,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,is_repeated_guest,is_weekend_arrival,arrival_date
0,Resort Hotel,BB,Direct,Direct,C,No Deposit,Transient,PRT,342,0,...,0,2.0,0,0,3,0,0.00,0,0,2015-07-01
1,Resort Hotel,BB,Direct,Direct,C,No Deposit,Transient,PRT,737,0,...,0,2.0,0,0,4,0,0.00,0,0,2015-07-01
2,Resort Hotel,BB,Direct,Direct,A,No Deposit,Transient,GBR,7,1,...,0,1.0,0,0,0,0,75.00,0,0,2015-07-01
3,Resort Hotel,BB,Corporate,Corporate,A,No Deposit,Transient,GBR,13,1,...,0,1.0,0,0,0,0,75.00,0,0,2015-07-01
4,Resort Hotel,BB,Online TA,TA/TO,A,No Deposit,Transient,GBR,14,2,...,0,2.0,0,0,0,0,98.00,0,0,2015-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,BB,Offline TA/TO,TA/TO,A,No Deposit,Transient,BEL,23,7,...,0,2.0,0,0,0,0,96.14,0,0,2017-08-30
119386,City Hotel,BB,Online TA,TA/TO,E,No Deposit,Transient,FRA,102,7,...,0,3.0,0,0,0,0,225.43,0,0,2017-08-31
119387,City Hotel,BB,Online TA,TA/TO,D,No Deposit,Transient,DEU,34,7,...,0,2.0,0,0,0,0,157.71,0,0,2017-08-31
119388,City Hotel,BB,Online TA,TA/TO,A,No Deposit,Transient,GBR,109,7,...,0,2.0,0,0,0,0,104.40,0,0,2017-08-31


In [15]:
# 저장
full = X.copy()
full["is_canceled"] = y
full.to_parquet(OUT / "dataset.parquet", index=False)
print("saved -> data/processed/dataset.parquet")

saved -> data/processed/dataset.parquet
